In [31]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

## Data Loading

In [ ]:


def load_adult_data():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    df = pd.read_csv(url, header=None)
    df.columns = [
        "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
        "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
        "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
    ]
    # Convert numeric Income to binary: 0 if <=50K, 1 if >50K
    df['Income'] = df['Income'].apply(lambda x: 0 if x.strip() == "<=50K" else 1)
    return df


def load_telco_churn_data():
    # https://www.kaggle.com/blastchar/telco-customer-churn/downloads/WA_Fn-UseC_-Telco-Customer-Churn.csv/1
    # df = pd.read_csv(r'C:\develop\data\telco-customer-churn\WA_Fn-UseC_-Telco-Customer-Churn.csv')
    df = pd.read_csv(f"datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv")
    train_cols = df.columns[1:-1] # First column is an ID
    label = df.columns[-1]
    X_df = df[train_cols]
    y_df = df[label] # 'Yes, No'
    y_df = y_df.apply(lambda x: 1 if x == 'Yes' else 0)
    dataset = {
        'problem': 'classification',
        'full': {
            'X': X_df,
            'y': y_df,
        },
    }
    
    return dataset


def load_credit_data():
    # https://www.kaggle.com/mlg-ulb/creditcardfraud
    # df = pd.read_csv(r'C:\develop\data\creditcardfraud\creditcard.csv')
    df = pd.read_csv(f"datasets/creditcard.csv")
    train_cols = df.columns[0:-1]
    label = df.columns[-1]
    X_df = df[train_cols]
    y_df = df[label]
    dataset = {
        'problem': 'classification',
        'full': {
            'X': X_df,
            'y': y_df,
        },
    }
    
    return dataset

## Custom DP-EBM Algorithm

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dp_gaussian_noise(scale, size=None):
    """Generate Gaussian noise with standard deviation 'scale'."""
    return np.random.normal(0, scale, size=size)

def bin_feature(x, n_bins=32):
    """
    For a 1D numpy array x, if numeric then bin using quantiles;
    otherwise (categorical) map each unique value to an index.
    
    Returns:
      bins: array of bin indices (integers)
      edges: if numeric, the bin edge values; if categorical, the sorted unique values.
    """
    # Check if numeric (or convertible to float)
    try:
        # If all elements can be cast to float, we assume numeric.
        x_float = x.astype(float)
        quantiles = np.linspace(0, 1, n_bins + 1)
        edges = np.quantile(x_float, quantiles)
        # Ensure edges are unique (if data is concentrated, there may be fewer bins)
        edges = np.unique(edges)
        bins = np.digitize(x_float, edges, right=False) - 1
        bins = np.clip(bins, 0, len(edges) - 2)
        return bins, edges
    except (ValueError, TypeError):
        # Otherwise treat as categorical.
        uniques = np.unique(x)
        # Sort the unique values (this defines an order)
        uniques = np.sort(uniques)
        # Map each category to its index in the sorted list.
        mapping = {val: i for i, val in enumerate(uniques)}
        bins = np.array([mapping[val] for val in x])
        return bins, uniques

# DP-EBM training
def dp_ebm_train(X, y, n_bins=32, n_epochs=300, learning_rate=0.01, epsilon=1.0, delta=1e-5):
    """
    A simplified DP-EBM training function that works on all columns.
    
    For numeric features the binning is done using quantiles;
    for non-numeric (categorical) features each unique value is assigned its own bin.
    
    The Gaussian noise scale is computed using the Gaussian mechanism calibration
    assuming sensitivity Δ = 1:
    
         noise_scale = (sqrt(2 * ln(1.25/delta))) / epsilon
    
    Parameters:
      X            : DataFrame with features (numeric or categorical).
      y            : Binary target as a 1D numpy array (0/1).
      n_bins       : Maximum number of bins for numeric features.
      n_epochs     : Number of boosting epochs (cycles over features).
      learning_rate: Step size for updating shape functions.
      epsilon      : Privacy budget.
      delta        : Failure probability.
      
    Returns:
      shape_functions: dict mapping feature names to learned arrays (one per bin).
      bin_info       : dict mapping feature names to (bins, edges) information.
      F              : Final additive prediction (before the sigmoid).
    """
    # Compute noise scale from privacy parameters (assume sensitivity Δ = 1)
    noise_scale = (np.sqrt(2 * np.log(1.25 / delta))) / epsilon

    features = X.columns
    bin_info = {}
    shape_functions = {}
    
    # Precompute bin assignments and initialize shape functions for every feature.
    for feature in features:
        x = X[feature].values
        bins, edges = bin_feature(x, n_bins=n_bins)
        bin_info[feature] = (bins, edges)
        # For numeric features, number of bins = len(edges) - 1; for categorical, = len(uniques)
        if np.issubdtype(np.array(edges).dtype, np.number):
            shape_functions[feature] = np.zeros(len(edges) - 1)
        else:
            shape_functions[feature] = np.zeros(len(edges))
    
    n = len(y)
    F = np.zeros(n)
    
    # Boosting loop: cycle through all features and update their shape functions.
    for epoch in range(n_epochs):
        for feature in features:
            x = X[feature].values
            # For numeric features, use np.digitize; for categorical, we use a mapping based on sorted edges.
            edges = bin_info[feature][1]
            if np.issubdtype(np.array(edges).dtype, np.number):
                x_numeric = x.astype(float)
                bins = np.digitize(x_numeric, edges, right=False) - 1
                bins = np.clip(bins, 0, len(shape_functions[feature]) - 1)
            else:
                mapping = {val: i for i, val in enumerate(edges)}
                bins = np.array([mapping[val] for val in x])
            
            f = shape_functions[feature]
            unique_bins = np.unique(bins)
            for b in unique_bins:
                idx = np.where(bins == b)[0]
                if len(idx) == 0:
                    continue
                residuals = y[idx] - F[idx]
                r_mean = np.mean(residuals)
                noisy_update = learning_rate * (r_mean + dp_gaussian_noise(noise_scale))
                f[b] += noisy_update
                F[idx] += noisy_update
            shape_functions[feature] = f
    return shape_functions, bin_info, F

# DP-EBM prediction
def dp_ebm_predict(X, shape_functions, bin_info):
    """
    Predict using the learned shape functions.
    
    For numeric features, bin using np.digitize;
    for categorical features, use the mapping defined by the sorted unique values.
    
    Returns the additive prediction F (before applying the sigmoid).
    """
    n = len(X)
    F = np.zeros(n)
    for feature in X.columns:
        x = X[feature].values
        edges = bin_info[feature][1]
        if np.issubdtype(np.array(edges).dtype, np.number):
            x_numeric = x.astype(float)
            bin_indices = np.digitize(x_numeric, edges, right=False) - 1
            bin_indices = np.clip(bin_indices, 0, len(shape_functions[feature]) - 1)
        else:
            mapping = {val: i for i, val in enumerate(edges)}
            bin_indices = np.array([mapping.get(val, 0) for val in x])
        F += shape_functions[feature][bin_indices]
    return F

## Adult Dataset

In [47]:
# Load Adult data
df = load_adult_data()

X = df.drop(columns=["Income"])
y = df["Income"].values.astype(float)

# Split data into train and test sets.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Set privacy parameters.
epsilon = 4.0   # privacy budget
delta = 1e-5    # failure probability

# Train DP-EBM on all columns.
shape_functions, bin_info, F_train = dp_ebm_train(X_train, y_train, n_bins=32, n_epochs=280, learning_rate=0.01, epsilon=epsilon, delta=delta)

# Evaluate on training data.
train_preds_raw = F_train
train_probs = sigmoid(train_preds_raw)
train_preds = (train_probs > 0.5).astype(int)

# print("Training Accuracy:", accuracy_score(y_train, train_preds))
print("Adult Dataset Training AUC:", roc_auc_score(y_train, train_probs))

# Predict on test data.
F_test = dp_ebm_predict(X_test, shape_functions, bin_info)
test_probs = sigmoid(F_test)
test_preds = (test_probs > 0.5).astype(int)

# print("Test Accuracy:", accuracy_score(y_test, test_preds))
print("Adult Dataset Test AUC:", roc_auc_score(y_test, test_probs))

Adult Dataset Training AUC: 0.8417610798846461
Adult Dataset Test AUC: 0.843542959565846


## Telco Churn Dataset

In [35]:
# Load Telco Churn data
dataset = load_telco_churn_data()
X = dataset['full']['X']
y = dataset['full']['y'].values.astype(float)

# Split data into train and test sets.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Set privacy parameters.
epsilon = 4.0   # privacy budget
delta = 1e-5    # failure probability

# Train the DP-EBM model.
shape_functions, bin_info, F_train = dp_ebm_train(X_train, y_train, n_bins=32, n_epochs=125, learning_rate=0.001, epsilon=epsilon, delta=delta)

# Evaluate training performance.
train_preds_raw = F_train
train_probs = sigmoid(train_preds_raw)
train_auc = roc_auc_score(y_train, train_probs)
print("Telco Churn Training AUC:", train_auc)

# Predict on the test set.
F_test = dp_ebm_predict(X_test, shape_functions, bin_info)
test_probs = sigmoid(F_test)
test_auc = roc_auc_score(y_test, test_probs)
print("Telco Churn Test AUC:", test_auc)


Telco Churn Training AUC: 0.9515702256121044
Telco Churn Test AUC: 0.8225063918763651


## Credit Card Dataset

In [55]:
# Load Credit Card Fraud data
dataset = load_credit_data()
X = dataset['full']['X']
y = dataset['full']['y'].values.astype(float)

# Split data into train and test sets.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Set privacy parameters.
epsilon = 4.0   # privacy budget
delta = 1e-5    # failure probability

# Train the DP-EBM model.
shape_functions, bin_info, F_train = dp_ebm_train(X_train, y_train, n_bins=32, n_epochs=500, learning_rate=0.01, epsilon=epsilon, delta=delta)

# Evaluate training performance.
train_preds_raw = F_train
train_probs = sigmoid(train_preds_raw)
train_auc = roc_auc_score(y_train, train_probs)
print("Credit Card Fraud Training AUC:", train_auc)

# Predict on the test set.
F_test = dp_ebm_predict(X_test, shape_functions, bin_info)
test_probs = sigmoid(F_test)
test_auc = roc_auc_score(y_test, test_probs)
print("Credit Card Fraud Test AUC:", test_auc)

Credit Card Fraud Training AUC: 0.8583815352699271
Credit Card Fraud Test AUC: 0.8754423371768516
